# Global search

ベースラインRAGは、答えを構成するためにデータセット全体の情報の集約を必要とするクエリに苦戦している。ベースラインRAGは、データセット内の意味的に類似したテキストコンテンツのベクトル検索に依存しているため、「データ内のトップ5のテーマは何か」といったクエリのパフォーマンスは最悪である。クエリには、正しい情報へ誘導するものは何もない。

しかし、GraphRAGを使えば、LLMが生成した知識グラフの構造から、データセット全体の構造（ひいてはテーマ）を知ることができるため、このような質問に答えることができる。これにより、プライベートデータセットを、あらかじめ要約された意味のあるセマンティッククラスタに整理することができる。LLMは、我々の大域的検索法を用いて、ユーザからのクエリに応答する際に、これらのクラスターを使ってこれらのテーマを要約する。

In [1]:
%load_ext dotenv

import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

ベースとなるLLMの指定

In [2]:
api_key = os.environ["OPENAI_API_KEY"]
llm_model = "gpt-4o"

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

`python -m graphrag.index --root ./ragtest`で生成された`.parquet`ファイルへのpathを指定

In [3]:
input_dir="ragtest/output/20240801-212257/artifacts"

community_report_table = "create_final_community_reports"
entity_table = "create_final_nodes"
entity_embedding_table = "create_final_entities"

`pd.DataFrame`として読み出し

In [4]:
report_df = pd.read_parquet(f"{input_dir}/{community_report_table}.parquet")
entity_df = pd.read_parquet(f"{input_dir}/{entity_table}.parquet")
entity_embedding_df = pd.read_parquet(f"{input_dir}/{entity_embedding_table}.parquet")

DataFrameからGraphRAGとして使える形に変換

In [5]:
reports = read_indexer_reports(report_df, entity_df, 2)
entities = read_indexer_entities(entity_df, entity_embedding_df, 2)

グローバルな文脈を構築

In [6]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

グローバルサーチのためのエンジンをインスタンス化

In [7]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [8]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

グローバルサーチ

In [9]:
result = await search_engine.asearch(
    "主人公の交友関係を説明して"
)

print(result.response)

### 主人公の交友関係

主人公であるリナとリオは、村の中で非常に重要な役割を果たしており、彼らの交友関係は村全体の調和と福祉に大きく寄与しています。

#### リナの役割と交友関係

リナは村の調停者として、村人たちの間で調和を促進する重要な役割を担っています。彼女は村の長老たちと深く関わり、歴史的な文脈を共有することで村の文化と伝統を守る努力をしています [Data: Reports (2)]。さらに、リナは石を使ってドラゴンとコミュニケーションを取る能力を持っており、村人とドラゴンの間の理解と協力を促進しています。この関係は村の調和を維持し、潜在的な紛争を解決するために重要です [Data: Reports (2, 1)]。

#### リオの役割と交友関係

リオは村のコミュニティ内で重要なリーダーであり、協力と知識の共有を強調しています。彼は村の文化遺産と言語を保存することに尽力し、村人たちが共同で活動に参加するよう促しています [Data: Reports (2)]。また、リオは環境の持続可能性に重点を置いており、村人たちが自然の復興を促進する活動に参加するよう奨励しています。この持続可能性への焦点は村の長期的な健康と資源の確保にとって重要です [Data: Reports (2)]。

#### リナとリオの兄弟関係

リナとリオは強い兄弟の絆を共有しており、村の中でのそれぞれの役割を強化しています。彼らのパートナーシップは相互支援と村の福祉へのコミットメントによって特徴付けられています [Data: Reports (2)]。

### 結論

リナとリオの交友関係は、村の調和と持続可能性を維持するために不可欠です。リナは調停者としての役割を果たし、リオはリーダーシップと環境保護に尽力しています。彼らの兄弟関係は、村全体の福祉に大きく貢献しています。


In [10]:
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,3,Mura and the Dragon's Legacy,1.0,# Mura and the Dragon's Legacy\n\nThe communit...,8.5
1,2,Community of Mura: Rina and Rio,1.0,# Community of Mura: Rina and Rio\n\nThe commu...,7.5
2,0,Dragon's Sand and Eld,0.4,# Dragon's Sand and Eld\n\nThe community cente...,6.5
3,1,Dragon and Lake Community,0.2,# Dragon and Lake Community\n\nThe community c...,8.0


In [11]:
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 2. LLM tokens: 4637
